## Path Planning

For the path planning component, we employed **cell decomposition** combined with the **Best-First Search** strategy, which led us to implement the A* algorithm.

### Disclaimer

The A* pathfinding algorithm was developed by following the pseudocode provided in the lecture *"Space Robotics Systems"*, delivered by Prof. Antonio Genova and Dr. Edoardo Del Vecchio from the Department of Mechanical and Aerospace Engineering, Sapienza University of Rome. During the development process, I used **ChatGPT** by OpenAI and **GitHub Copilot** to help refine the implementation, improve the code structure, and add meaningful comments. These tools provided suggestions that I adapted to meet the specific requirements of this project. This acknowledgment ensures transparency and helps prevent concerns related to plagiarism.


### Implementation Overview
To implement the A* algorithm, we followed this pseudocode:

<img src="pseudocode_Astar.jpg" alt="A* Pseudocode" style="width:40%; height:auto;">

### Importing Required Libraries

In [1]:
import math
import numpy as np
import heapq
from PIL import Image, ImageDraw

### NODE Class Overview

The `NODE` class is designed to represent a point in the grid, encapsulating all the necessary attributes and methods for implementing the A* pathfinding algorithm. Below, we describe the key components of the `NODE` class and their roles:

#### **Attributes and Methods**

- `position`: Represents the coordinates of the node on the grid (e.g., `[x, y]`).

- `g`: The **cost** from the starting node to the current node. It is initialized to infinity (`float('inf')`) to indicate that nodes start off as unexplored.

- `h`: The **heuristic estimate** of the cost from the current node to the goal. This value guides the A* algorithm toward the goal.

- `f`: The **total cost**, computed as `f = g + h`. This value is used to prioritize nodes during the search.

- `parent`: A reference to the **parent node** in the path, used for path reconstruction.

- **Comparison (`__lt__`)**:
This method allows nodes to be compared based on their f value. It is crucial because we use heapq, a priority queue that requires comparison for efficient sorting.

- **String Representation (`__repr__`)**:
This method defines how the node should be represented when printed.

- **Path Reconstruction (`reconstruct_path`)**:
This static method is used to reconstruct the optimal path once the goal is reached. Starting from the goal node, it traces back through each node’s parent attribute until it reaches the start, thereby creating the complete path.



In [2]:
class NODE:
    def __init__(self, position, g=float('inf'), h=0, parent=None):
        self.position = position
        self.g = g
        self.h = h
        self.f = g + h
        self.parent = parent

    # Nodes will be compared based on their f value, we use this because of the heapq
    def __lt__(self, other):
        return self.f < other.f

    # Function to represent the node
    def __repr__(self):
        return f"Node(position={self.position}, g={self.g}, h={self.h}, f={self.f})"

    @staticmethod
    def reconstruct_path(node):
        path = []
        while node:
            path.append(node.position)
            node = node.parent
        return path[::-1]

### Heuristic Function: Octile Distance

The heuristic function used in this implementation is based on the **Octile distance**, which is suitable when diagonal movement is allowed (resulting in 8 possible movement directions). This heuristic ensures an efficient estimation of the shortest path by considering both orthogonal and diagonal moves.
The values assigned to these movements are:
- Vertical/Horizontal Movement Cost = 1
- Diagonal Movement Cost = $\sqrt{2} \approx 1.414$


In [3]:
# Heuristic function using Octile distance (allowing 8 directions of movement)
def Heuristic_function(node, goal):
    dx = abs(node.position[0] - goal.position[0])
    dy = abs(node.position[1] - goal.position[1])
    return max(dx, dy) + (math.sqrt(2) - 1) * min(dx, dy)

### A* Pathfinding Algorithm

The `A_star` function is the core of our path planning, implementing the A* search algorithm to find an optimal path from a start to a goal position on a grid-based map. Below, we provide a breakdown of how the algorithm operates:

#### Explanation of the Key Steps

- **Initialization**:
  - **Start and Goal Nodes**: The function begins by creating a start node (`START_NODE`) with `g = 0` (the cost to reach the start node is zero) and a goal node (`GOAL_NODE`).
  - **Heuristic Calculation**: The heuristic (`h`) for the start node is computed using the `Heuristic_function`.
  - **Data Structures**:
    - **Open List** (`OPEN_list`): A priority queue that stores nodes yet to be explored, sorted based on their `f` value (total estimated cost).
    - **Closed Set** (`CLOSED_set`): A set that tracks nodes that have already been explored.
    - **Nodes Dictionary** (`nodes`): Tracks nodes that have already been visited, ensuring that if a node is revisited, the algorithm always uses the shortest known path to reach it rather than simply using the most recent path.

- **Environment Matrix Representation**:
  - The **environment map** is represented as a matrix where:
    - **`0`** indicates a **free path** that the algorithm can traverse.
    - **`1`** indicates an **obstacle** that cannot be crossed.

- **Search Loop**:
  - The algorithm proceeds until the **open list** is empty. It repeatedly pops the node with the **lowest `f` value** from the open list.
  - If the **current node** is already in the closed set, it is skipped to avoid redundant processing. Otherwise, it is added to the closed set for tracking purposes.

- **Goal Check**:
  - If the current node's position matches the goal, the function reconstructs the **optimal path** using the `reconstruct_path()` method.

- **Exploration of Neighboring Nodes**:
  - **Moves**: The possible moves include 8 directions — vertical, horizontal, and diagonal.
  - For each potential move:
    - The **neighbor's position** is calculated, and checks are performed to ensure that it is within bounds and is not an obstacle.
    - If the position is already in the closed set, it is skipped.
    - The **movement cost** is calculated:
      - **Vertical/Horizontal Movement Cost** = 1
      - **Diagonal Movement Cost** = $\sqrt{2} \approx 1.414$
  - The **tentative cost (`g_tentative`)** to reach the neighbor is computed by adding the movement cost to the `g` value of the current node.

- **Neighbor Node Handling**:
  - If the neighbor node does not exist in the `nodes` dictionary, a **new node** is created.
  - If the newly found path to the neighbor is **more efficient** than any previously known path (i.e., has a lower `g` value), the neighbor node's values are updated to reflect this improvement.
  - The neighbor node is then added to the **open list** for future exploration.

- **No Path Found**:
  - If the **open list** is exhausted without reaching the goal, the function returns **`None`** and prints "No path found."

In [4]:
def A_star(environment_map, start_position, goal_position):

    # Create the start and goal nodes
    start_node = NODE(start_position, g=0)
    goal_node = NODE(goal_position)

    # Set the heuristic value for the start node
    start_node.h = Heuristic_function(start_node, goal_node)
    start_node.f = start_node.g + start_node.h

    # Initialize the open and closed lists
    open_list = []
    heapq.heappush(open_list, start_node)
    closed_set = set()

    # Dictionary to keep track of nodes
    nodes = {}
    nodes[tuple(start_node.position)] = start_node

    while open_list:
        # Pop the node with the lowest f value
        current_node = heapq.heappop(open_list)

        # If the current node is in the closed set, skip it
        if tuple(current_node.position) in closed_set:
            continue

        # Add the current node's position to the closed set
        closed_set.add(tuple(current_node.position))

        # If the current node is the goal, reconstruct the path
        if current_node.position == goal_node.position:
            path = NODE.reconstruct_path(current_node)
            return path

        # Possible moves: 8 directions (including diagonals)
        moves = [[1, 0], [0, 1], [-1, 0], [0, -1],
                 [1, 1], [-1, -1], [1, -1], [-1, 1]]

        # Explore neighbors
        for move in moves:
            neighbor_position = [current_node.position[0] + move[0],
                                 current_node.position[1] + move[1]]
            neighbor_pos = tuple(neighbor_position)

            # Skip if out of bounds or obstacle
            if (neighbor_position[0] < 0 or neighbor_position[0] >= environment_map.shape[0] or
                neighbor_position[1] < 0 or neighbor_position[1] >= environment_map.shape[1] or
                environment_map[neighbor_position[0], neighbor_position[1]] == 1):
                continue

            # Skip if in closed set
            if neighbor_pos in closed_set:
                continue

            # Calculate movement cost (diagonal movement cost is sqrt(2))
            dx = abs(move[0])
            dy = abs(move[1])
            movement_cost = math.sqrt(2) if dx == 1 and dy == 1 else 1

            # Calculate tentative g value
            g_tentative = current_node.g + movement_cost

            # Create or get the neighbor node
            if neighbor_pos not in nodes:
                neighbor_node = NODE(neighbor_position)
                nodes[neighbor_pos] = neighbor_node
            else:
                neighbor_node = nodes[neighbor_pos]

            # If this path to neighbor is better, record it
            if g_tentative < neighbor_node.g:
                neighbor_node.g = g_tentative
                neighbor_node.h = Heuristic_function(neighbor_node, goal_node)
                neighbor_node.f = neighbor_node.g + neighbor_node.h
                neighbor_node.parent = current_node

                # Add the neighbor to the open list
                heapq.heappush(open_list, neighbor_node)

    print("No path found.")
    return None


### Example: Finding the Treasure

To illustrate how our A* pathfinding algorithm works, we present a simple example: the goal is to reach the **treasure** in the shortest possible path.

In the following grid map:

- The **starting point** is marked by a **pirate boat**, and the **goal** is to reach the **treasure**.
- The algorithm efficiently navigates through free paths while avoiding obstacles to determine the optimal route.

Below is an illustration of the grid:

<img src="maze_map_example.png" alt="Maze map" style="width:40%; height:auto;">


In [5]:
def convert_image_to_binary_array(image_path):
    # Load the image
    image = Image.open(image_path)

    # Convert the image to grayscale
    image = image.convert("L")

    # Set the binary threshold
    binary_threshold = 200

    # Convert grayscale to binary image
    bw_image = image.point(lambda p: p < binary_threshold and 1)

    # Convert image to numpy array
    binary_matrix = np.array(bw_image)
    return binary_matrix

def swap_path_coordinates(path_coordinates):
    # Swap the x and y coordinates of the path (to match the image)
    swapped_path = [[y, x] for x, y in path_coordinates]
    return swapped_path

def display_path(maze_name, path):
    # Load the maze image
    maze_path = maze_name
    maze_img = Image.open(maze_path)

    # Convert the maze image to RGB (to draw in color)
    maze_img = maze_img.convert("RGB")

    # Swap the x and y coordinates of the path (to match the image)
    path = swap_path_coordinates(path)

    # Create a drawing object
    draw = ImageDraw.Draw(maze_img)

    # Draw the path (red line or points)
    for i in range(len(path) - 1):
        x1, y1 = path[i]
        x2, y2 = path[i + 1]
        draw.line((x1, y1, x2, y2), fill=(255, 0, 0), width=4)  # Red line with width 2

    # Save and show the image
    maze_img.save("maze_with_path.png")
    maze_img.show()
    return

# Test the A* algorithm

# Define the start and goal positions
start_position = [340, 120]
goal_position = [310, 793]

# Load the binary matrix from the image
binary_matrix = convert_image_to_binary_array("maze_map_example.png")

# Find the path using A*
path = A_star(binary_matrix, start_position, goal_position)

# Display the path on the maze image
display_path("maze_map_example.png", path)

### Result

The resulting path is shown below, marked in **red** to indicate the shortest route taken by the pirate boat to reach the treasure.

<img src="maze_with_path.png" alt="maze with path" style="width:40%; height:auto;">